In [26]:
import logging
from typing import Annotated

import typer
from dotenv import load_dotenv
from metagame import TradingClient
from metagame.websocket_api import Side

from os import getenv

JWT = getenv("JWT")
API_URL = getenv("API_URL") 
ACT_AS = int(getenv("ACT_AS", None))


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
load_dotenv()

# app = typer.Typer(pretty_exceptions_show_locals=False)


ETFS = {
    'ghi_tw': {
        'golf_tw': 3,
        'hotel_tw': 2,
        'india_tw': 1
    },
    'def_tw': {
        'delta_tw': 1,
        'echo_tw': 1,
        'foxtrot_tw': 4
    },
    'abc_tw': {
        'alpha_tw': 2,
        'bravo_tw': 2,
        'charlie_tw': 2
    }
}
ALL_MARKETS = list(set(ETFS.keys()).union(*[set(etf_constituents.keys()) for etf_constituents in ETFS.values()]))

# def main(
#     size: float = 100.0,
# ):
    # with TradingClient(api_url, jwt, act_as) as client:
    #     arb_bot(
    #         client,
    #         market_name=market_name,
    #         size=size,
    #     )


def arb_bot(
    client: TradingClient,
    *,
    size: float,
):
    global ETFS
    state = client.state()
    market_name_to_id = state.market_name_to_id
    market_data = {}
    for market_name in ALL_MARKETS:
        market_id = market_name_to_id[market_name]
        market = state.markets.get(market_id)
        if not market:
            logger.info(f"No market data available for market {market_name}")
            continue
        market_data[market_name] = market
    

    # market_name = markets[0]
    # state = client.state()
    # market_id = state.market_name_to_id[market_name]
    # market = state.markets.get(market_id)
    # if not market:
    #     logger.info(f"No market data available for market {market_name}")
    #     return

    # min_settlement = market.definition.min_settlement
    # max_settlement = market.definition.max_settlement

    # client.create_order(market_id, min_settlement, size, Side.BID)
    # client.create_order(market_id, max_settlement, size, Side.OFFER)


# if __name__ == "__main__":
#     app()


In [ ]:
import logging
from typing import Annotated
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
load_dotenv()

In [37]:


# import typer
from dotenv import load_dotenv
from metagame import TradingClient
from metagame.websocket_api import Side
from typing import Literal
from metagame.websocket_api import CancelOrder, ClientMessage, CreateOrder, Side


from os import getenv

JWT = getenv("JWT")
API_URL = getenv("API_URL") 
# ACT_AS = int(getenv("ACT_AS", None))
ACT_AS=106 # arrbot



# app = typer.Typer(pretty_exceptions_show_locals=False)

import time

ETFS = {
    'ghi_tw': {
        'golf_tw': 3,
        'hotel_tw': 2,
        'india_tw': 1
    },
    'def_tw': {
        'delta_tw': 1,
        'echo_tw': 1,
        'foxtrot_tw': 4
    },
    'abc_tw': {
        'alpha_tw': 2,
        'bravo_tw': 2,
        'charlie_tw': 2
    }
}

ALL_MARKETS = list(set(ETFS.keys()).union(*[set(etf_constituents.keys()) for etf_constituents in ETFS.values()]))

    
    
# offers = [
    #     ClientMessage(
    #         create_order=CreateOrder(
    #             market_id=
    #         )
    #     )
    # ]







# for etf in etfs:





# for etf_bid in [True, False]:



In [ ]:
import time
def get_price_size(market, market_type: Literal['etf', 'stock'], considering_etf_bid: bool):
    '''returns best price, size. Returns None, 0 if no contract available'''
    assert market_type in ['etf', 'stock']
    considering_bid = considering_etf_bid if market_type == 'etf' else not considering_etf_bid
    if considering_bid:
        if len(market.offers) == 0:
            return None, 0
        return market.offers[0].price, market.offers[0].size
    else:
        if len(market.bids) == 0:
            return None, 0
        return market.bids[0].price, market.bids[0].size
# MAX_TRADES = 5
while True:
    time.sleep(1)
    with TradingClient(API_URL, JWT, ACT_AS) as client:
        state = client.state()
        market_name_to_id = state.market_name_to_id

        name_to_market = {}
        for market_name in ALL_MARKETS:
            market_id = state.market_name_to_id[market_name]
            name_to_market[market_name] = state.markets[market_id]

        for etf_name in ETFS:
        # etf_name = 'abc_tw'
            etf_constituents = ETFS[etf_name]
            etf_stock_names = list(etf_constituents.keys())
            etf_market = name_to_market[etf_name]

            for considering_etf_bid in [True, False]:
                # get etf contracts (bid by default)
                etf_price, etf_size = get_price_size(market=etf_market, market_type='etf', considering_etf_bid=considering_etf_bid)

                unbundled_price = 0
                etf_deal_size = etf_size # the bottleneck of the deal

                stock_prices = {}
                for stock_name in etf_stock_names:
                    stock = name_to_market[stock_name]
                    price, size = get_price_size(market=stock, market_type='stock', considering_etf_bid=considering_etf_bid)

                    stock_weight = etf_constituents[stock_name]

                    # etf_weight_deal_size is the size of the best bid/offer, weighted by the stock weight
                    etf_weight_deal_size = size/stock_weight
                    etf_deal_size = min(etf_deal_size, etf_weight_deal_size)
                    
                    unbundled_price += stock_weight*price

                    stock_prices[stock_name] = price


                bundled_price = etf_price
                if (considering_etf_bid and etf_price < unbundled_price) or (not considering_etf_bid and etf_price > unbundled_price):
                    # # do the deal
                    # orders = []
                    # for stock_name in etf_stock_names:
                    #     orders.append(
                    #         ClientMessage(
                    #             create_order=CreateOrder(
                    #             market_id=market_name_to_id[stock_name],
                    #             price=stock_prices[stock_name],
                    #             size=etf_deal_size*etf_constituents[stock_name],
                    #             side=Side.OFFER if considering_etf_bid else Side.BID,
                    #             )
                    #     ))
                    # orders.append(
                    #     ClientMessage(
                    #         create_order=CreateOrder(
                    #             market_id=market_name_to_id[etf_name],
                    #             price=(bundled_price + unbundled_price)/2,
                    #             size=etf_deal_size,
                    #             side=Side.BID if considering_etf_bid else Side.OFFER,
                    #         )
                    #     )
                    # )
                    s = f'''Considering doing this deal:
{"ETF BID" if considering_etf_bid else "ETF OFFER"}

ETF
{etf_name}: {etf_price}

STOCKS
'''
                    for stock_name in etf_stock_names:
                        s += f"{stock_name}: {stock_prices[stock_name]}\n"
                    
                    s += f'''\nBUNDLED PRICE: {bundled_price}
UNBUNDLED PRICE: {unbundled_price}

DEAL SIZE: {etf_deal_size}
'''

                    if considering_etf_bid:
                        assert etf_price < unbundled_price
                    else:
                        assert etf_price > unbundled_price

                    # logger.info(f"Placing {len(orders)} orders")
                    # MAX_TRADES -= 1
                    # if MAX_TRADES <= 0:
                    #     assert False
                    # try:
                    #     client.request_many(orders)
                    #     time.sleep(0.5)

                    # except Exception as e:
                    #     logger.error(f"Error placing orders: {e}")
                    #     continue
                else:
                    logger.info(f"No deal to do in {etf_name}")
                    continue




INFO:__main__:No deal to do in ghi_tw
INFO:__main__:No deal to do in def_tw
INFO:__main__:No deal to do in abc_tw
INFO:__main__:No deal to do in abc_tw
INFO:__main__:No deal to do in ghi_tw
INFO:__main__:No deal to do in def_tw
INFO:__main__:No deal to do in abc_tw
INFO:__main__:No deal to do in abc_tw
INFO:__main__:No deal to do in ghi_tw
INFO:__main__:No deal to do in def_tw
INFO:__main__:No deal to do in abc_tw
INFO:__main__:No deal to do in abc_tw
INFO:__main__:No deal to do in ghi_tw
INFO:__main__:No deal to do in def_tw
INFO:__main__:No deal to do in abc_tw
INFO:__main__:No deal to do in abc_tw
INFO:__main__:No deal to do in ghi_tw
INFO:__main__:No deal to do in def_tw
INFO:__main__:No deal to do in abc_tw
INFO:__main__:No deal to do in abc_tw
INFO:__main__:No deal to do in ghi_tw
INFO:__main__:No deal to do in def_tw
INFO:__main__:No deal to do in abc_tw
INFO:__main__:No deal to do in abc_tw
INFO:__main__:No deal to do in ghi_tw
INFO:__main__:No deal to do in def_tw
INFO:__main_

In [ ]:
!pip3 install dotenv